<a href="https://colab.research.google.com/github/Ummhabibi/MJ/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [3]:
dataset, info = tfds.load("tf_flowers",
                          as_supervised=True,
                          with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [4]:
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info. features["label"].num_classes

In [5]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load("tf_flowers", split=["train[:10%]", "train[10%:25%]", "train[25%:]"], as_supervised=True)

In [6]:
def preprocess(image,label):
    resized_image = tf.image.resize(image, [224,224])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return final_image, label


In [7]:
batch_size = 32
train_set = train_set_raw.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [8]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet",
                                            include_top=False)


avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)


model = tf.keras.Model(inputs=base_model.input, outputs=output)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [9]:
for layer in base_model.layers:
  layer.trainable = False

In [10]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.2, momentum=0.9)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model.fit(train_set, epochs=3, validation_data=valid_set)

Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


86/86 ━━━━━━━━━━━━━━━━━━━━ 35s 217ms/step - accuracy: 0.7100 - loss: 1.4696 - val_accuracy: 0.8330 - val_loss: 1.2883
Epoch 2/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 12s 139ms/step - accuracy: 0.8747 - loss: 0.8074 - val_accuracy: 0.8167 - val_loss: 1.5476
Epoch 3/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 142ms/step - accuracy: 0.9042 - loss: 0.6029 - val_accuracy: 0.8512 - val_loss: 1.2273


In [12]:
for layer in base_model.layers:
  layer.trainable = True

In [14]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01,
                                    momentum=0.9,)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(train_set,
                    epochs=6,
                    validation_data=valid_set)

Epoch 1/6
86/86 ━━━━━━━━━━━━━━━━━━━━ 79s 497ms/step - accuracy: 0.8136 - loss: 0.6539 - val_accuracy: 0.7913 - val_loss: 1.1561
Epoch 2/6
86/86 ━━━━━━━━━━━━━━━━━━━━ 40s 467ms/step - accuracy: 0.9693 - loss: 0.1212 - val_accuracy: 0.9093 - val_loss: 0.3542
Epoch 3/6
86/86 ━━━━━━━━━━━━━━━━━━━━ 42s 483ms/step - accuracy: 0.9857 - loss: 0.0404 - val_accuracy: 0.9038 - val_loss: 0.3753
Epoch 4/6
86/86 ━━━━━━━━━━━━━━━━━━━━ 82s 489ms/step - accuracy: 0.9918 - loss: 0.0220 - val_accuracy: 0.9256 - val_loss: 0.3544
Epoch 5/6
86/86 ━━━━━━━━━━━━━━━━━━━━ 82s 489ms/step - accuracy: 0.9959 - loss: 0.0178 - val_accuracy: 0.9220 - val_loss: 0.3667
Epoch 6/6
86/86 ━━━━━━━━━━━━━━━━━━━━ 82s 488ms/step - accuracy: 0.9976 - loss: 0.0103 - val_accuracy: 0.9147 - val_loss: 0.4464


In [15]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_set, verbose=2)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


12/12 - 7s - 560ms/step - accuracy: 0.9210 - loss: 0.3039
Test Accuracy: 92.10%


In [16]:
predictions = model.predict(test_set)



12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 315ms/step


In [17]:
import numpy as np

predicted_classes = np.argmax(predictions, axis=1)


In [18]:
print(predicted_classes)

[2 1 3 3 3 0 0 0 0 1 2 2 4 1 2 1 4 4 2 2 0 0 0 2 0 3 0 1 1 1 2 1 0 0 1 1 0
 3 0 4 0 3 0 4 3 3 2 4 0 1 2 1 3 0 1 4 1 2 2 3 0 0 4 0 1 1 0 1 2 4 3 3 0 3
 2 2 1 0 3 2 0 0 4 3 0 3 3 2 0 2 1 4 0 3 0 3 1 4 0 4 4 4 2 0 4 2 3 4 0 0 4
 4 4 2 2 4 2 3 3 0 3 2 1 1 2 1 4 3 0 2 0 2 4 4 0 3 0 0 1 2 0 1 4 3 1 2 2 4
 3 3 4 0 4 0 4 0 2 2 1 0 2 4 0 0 0 1 1 0 4 1 4 4 3 1 0 1 2 3 4 0 2 3 2 4 1
 0 0 4 4 2 0 3 4 0 0 4 3 2 0 2 2 3 4 0 1 4 1 0 1 2 1 2 2 2 1 1 4 2 4 1 0 0
 4 2 1 0 3 2 1 2 3 1 3 0 0 2 3 4 0 1 2 4 0 3 1 0 1 2 4 0 4 4 2 3 0 2 1 2 2
 4 1 2 4 3 0 0 0 2 4 0 2 2 2 2 0 3 1 4 0 3 3 3 2 0 3 3 4 4 4 4 2 3 3 3 0 0
 2 0 4 1 0 0 4 3 0 0 0 2 0 2 2 1 3 0 0 3 3 0 2 4 4 2 0 0 0 0 2 0 1 0 2 4 0
 2 0 0 1 0 1 0 4 1 1 3 1 4 2 4 0 3 3 4 0 3 2 4 3 1 1 2 4 0 1 2 0 3 3]


In [20]:
model.save('my_model.keras')  # Save the model in HDF5 format
